In [1]:
import torch
torch.cuda.empty_cache()
from datasets import load_dataset
from transformers import AutoConfig, AutoTokenizer, AutoModelForSequenceClassification,\
    TrainingArguments, Trainer, pipeline, DataCollatorWithPadding, set_seed
import evaluate
import numpy as np
import pandas as pd
import torch.nn as nn
import csv
from tqdm import tqdm
import matplotlib.pyplot as plt

In [2]:
set_seed(42)
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cuda:0'

In [3]:
#Get the dataset
snli = load_dataset("snli")

Found cached dataset snli (C:/Users/zebzi/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b)


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
# per chatGPT's suggestion, let's concatenate premise + hypothesis into a "text" column.
train_text = np.char.add(np.char.add(snli['train']['premise'], ' '), snli['train']['hypothesis'])
test_text = np.char.add(np.char.add(snli['test']['premise'], ' '), snli['test']['hypothesis'])
valid_text = np.char.add(np.char.add(snli['validation']['premise'], ' '), snli['validation']['hypothesis'])

# let's do a sanity check to make sure concatenation was done correctly
print(f'--train_text[0]--: {train_text[0]}')
print(f'--original + hypothesis--: {snli["train"]["premise"][0]+snli["train"]["hypothesis"][0]}\n')

print(f'--test_text[0]--: {test_text[0]}')
print(f'--original + hypothesis--: {snli["test"]["premise"][0]+snli["test"]["hypothesis"][0]}\n')

print(f'--valid_text[0]--: {valid_text[0]}')
print(f'--original + hypothesis--: {snli["validation"]["premise"][0]+snli["validation"]["hypothesis"][0]}')

snli["train"] = snli["train"].remove_columns(["premise", "hypothesis"])
snli["train"] = snli["train"].add_column("text", train_text)

snli["test"] = snli["test"].remove_columns(["premise", "hypothesis"])
snli["test"] = snli["test"].add_column("text", test_text)

snli["validation"] = snli["validation"].remove_columns(["premise", "hypothesis"])
snli["validation"] = snli["validation"].add_column("text", valid_text)

--train_text[0]--: A person on a horse jumps over a broken down airplane. A person is training his horse for a competition.
--original + hypothesis--: A person on a horse jumps over a broken down airplane.A person is training his horse for a competition.

--test_text[0]--: This church choir sings to the masses as they sing joyous songs from the book at a church. The church has cracks in the ceiling.
--original + hypothesis--: This church choir sings to the masses as they sing joyous songs from the book at a church.The church has cracks in the ceiling.

--valid_text[0]--: Two women are embracing while holding to go packages. The sisters are hugging goodbye while holding to go packages after just eating lunch.
--original + hypothesis--: Two women are embracing while holding to go packages.The sisters are hugging goodbye while holding to go packages after just eating lunch.


In [5]:
snli = snli.filter(lambda example: example["label"] != -1)
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
# tokenizer.to('cuda')

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)
    
tokenized_snli= snli.map(preprocess_function, batched=True)

Loading cached processed dataset at C:\Users\zebzi\.cache\huggingface\datasets\snli\plain_text\1.0.0\1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b\cache-a7e3f6a12dd5470b.arrow
Loading cached processed dataset at C:\Users\zebzi\.cache\huggingface\datasets\snli\plain_text\1.0.0\1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b\cache-10263cd2a97d376b.arrow
Loading cached processed dataset at C:\Users\zebzi\.cache\huggingface\datasets\snli\plain_text\1.0.0\1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b\cache-7dd2e0b9c4e30d43.arrow
Loading cached processed dataset at C:\Users\zebzi\.cache\huggingface\datasets\snli\plain_text\1.0.0\1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b\cache-6a07a2c8d2f1ad8c.arrow
Loading cached processed dataset at C:\Users\zebzi\.cache\huggingface\datasets\snli\plain_text\1.0.0\1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b\cache-aa9d2d1182cd7304.arrow
Loading cached processed 

In [6]:
# Load the model that was trained in HW1
id2label = {0: "entailment", 1: "neutral", 2:"contradiction"}
label2id = {"entailment": 0, "neutral": 1, "contradiction": 2}

config = AutoConfig.from_pretrained("config.json")
model = AutoModelForSequenceClassification.from_pretrained("pytorch_model.bin", config=config)
model.to(device)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [122]:
# Load the pipeline for natural language inference tasks
nlp = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, device=device)
# tokenized_snli["test"]["text"][0]
count = 0
i = 0

rowsList = []
while(count < 50):
    try:
        temp = snli["test"]["text"][i+100].split(".")
        # print(tokenized_snli["test"]["text"][i + 100])
        premise = temp[0]
        hypothesis = temp[1]
    except:
        print("no period\n")

    outputs = tokenizer(snli["test"]["text"][i+100], return_tensors='pt', padding=True, truncation=True)
    outputs = outputs.to('cuda:0')
    result = nlp.model(**outputs, output_hidden_states=True)
    # result = nlp(tokenized_snli["test"]["text"][i + 100])[0]'ArithmeticError

    # print(result["logits"].item())
    predicted_token_class_ids = result.logits.argmax(-1)
    predicted_label = nlp.model.config.id2label[predicted_token_class_ids.item()]
    # dumb = result.logits.detach().cpu()
    # dumb = np.array((result.logits[0][0].item(), result.logits[0][1].item(), result.logits[0][2].item()))
    probability = np.exp(result.logits[0].detach().cpu().numpy()) / np.sum(np.exp(result.logits[0].detach().cpu().numpy()))
    # probability = np.exp(dumb) / np.exp(np.sum(dumb))
    # print(probability)

    # print(result["hidden_states"][12].shape)
    temp = []
    # print(f"predicted_label: {predicted_label} and actual label: {id2label[tokenized_snli['test']['label'][i+100]]} with confidence: {probability[tokenized_snli['test']['label'][i+100]]:.3f}")
    if(predicted_label != id2label[tokenized_snli["test"]["label"][i+100]]):
        count += 1
        print()
        print("Premise: ",end="")
        print(premise)
        print("Hypothesis: ",end="")
        print(hypothesis)
        print(f"Confidence: {1 - probability[tokenized_snli['test']['label'][i+100]]:.3f}")
        print("The prediction is: ", predicted_label)
        print("The correct is: ", id2label[tokenized_snli["test"]["label"][i+100]])
        # temp = [str(count), f"Premise: {premise} \nHypothesis: {hypothesis}", id2label[tokenized_snli["test"]["label"][i+100]], f"Confidence: {result['score']:.3f} \n" + result['label'], str(len(premise) + len(hypothesis))]
        # rowsList.append(temp)
    i += 1

# with open("hw5CSV.csv", mode='w', newline='') as file:
#     writer = csv.writer(file)
#     writer.writerow(["", "Input Text", "Ground-Truth Label", "Predicted Label", "Input Text Length"])
#     writer.writerows(rowsList)
# print("Here is i: ", i)


Premise: A Ford car is making a right turn as 3 males are walking across the street behind the car
Hypothesis:  A sedan was turning a corner as walkers were crossing
Confidence: 0.027
The prediction is:  entailment
The correct is:  neutral

Premise: Three boys in white shirts are walking behind an older model Ford car
Hypothesis:  The boys are on the street
Confidence: 0.159
The prediction is:  neutral
The correct is:  entailment

Premise: Three girls blow out the candles of a cake made of Peeps
Hypothesis:  There are peeps in the garden
Confidence: 0.261
The prediction is:  neutral
The correct is:  contradiction

Premise: A crowd of people looking up at 3 people on the edge of the roof of a building
Hypothesis:  The crowd on the ground is watching 3 people on the roof's edge
Confidence: 0.042
The prediction is:  neutral
The correct is:  entailment

Premise: A person with a purple shirt is painting an image of a woman on a white wall
Hypothesis:  A woman paints a portrait of a person
